In [1]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 

In [2]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler
import logging
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
import statistics
import torch.optim.lr_scheduler as lr_scheduler
from scipy.special import softmax
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset


In [3]:
logging.getLogger('mne').setLevel(logging.WARNING)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import import_ipynb
#from Model import net
from Dataloader2 import EEGDataset

importing Jupyter notebook from Dataloader2.ipynb


In [4]:
def Normalize(raw):
    normals = []
    scaler = StandardScaler()
    for idx in range(len(raw)):
        normals.append(scaler.fit_transform(raw[idx]))

    return raw


In [5]:


# Set the directory path containing the folders
data_dir = r"C:\Users\admin\Desktop\MNE_Data"
dataset = 0
labels = []
# Get a list of all folders in the directory
folders = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]
v = 0
# Loop through each folder
for folder in folders:
    folder_path = os.path.join(data_dir, folder)
    files = [f for f in os.listdir(folder_path) if f.endswith('.fif')]
    print(folder)

    for file in files:
        file_path = os.path.join(folder_path, file)

        raw = mne.read_epochs(file_path).get_data(picks='eeg');
        #raw = Normalize(raw)


        if folder == 'FOCAL':
            labels.append(np.zeros((raw.shape[0], 1)))

        elif folder == 'IGE':
            labels.append(np.ones((raw.shape[0], 1)))

        elif folder == 'PNES':
            labels.append(np.ones((raw.shape[0], 1)) * 2)

        elif folder == 'TLE':
            labels.append(np.ones((raw.shape[0], 1)) * 3)
        

FOCAL
IGE
PNES
TLE


In [6]:
labels = np.array(labels)
freq = np.concatenate(labels).ravel()

In [7]:
unique_elements, element_counts = np.unique(freq, return_counts=True)
for element, count in zip(unique_elements, element_counts):
    print(f"Element {element}: Count {count}")

Element 0.0: Count 5916
Element 1.0: Count 14105
Element 2.0: Count 11262
Element 3.0: Count 7458
